In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

/home/hexagon/miniconda3/envs/torch39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class VAE(nn.Module):
    def __init__(self, latent):
        super(VAE, self).__init__()
        self.k = latent
        self.mu = nn.Linear(2*self.k, self.k)
        self.var = nn.Linear(2*self.k, self.k)

        encoder = []
        for i, o in [(3, 24), (24, 36)]:
            encoder.append(nn.Conv2d(i, o, kernel_size=5, stride=2, padding=0))
            encoder.append(nn.ReLU())
        for i, o in [(36, 48), (48, 64), (64, 64)]:
            if i == 36:
                encoder.append(nn.Conv2d(i, o, kernel_size=5, stride=2, padding=0))
            else:
                encoder.append(nn.Conv2d(i, o, kernel_size=3, stride=1, padding=0))
            encoder.append(nn.ReLU())
        encoder.append(nn.Flatten())
        for i, o in [(1152, 1000), (1000, 100), (100, 2*self.k)]:
            encoder.append(nn.Linear(i, o))
            if i == 1152 or i == 1000:
                encoder.append(nn.ReLU())
        self.encoder = nn.Sequential(*encoder)

        decoder = []
        for i, o in [(self.k, 2*self.k), (2*self.k, 100), (100, 1000), (1000, 1152)]:
            decoder.append(nn.Linear(i, o))
            if i == 100 or i == 1000:
                decoder.append(nn.ReLU())
        decoder.append(nn.Unflatten(1, (64, 1, 18)))
        for i, o in [(64, 64), (64, 48), (48, 36)]:
            if i == 36:
                decoder.append(nn.ConvTranspose2d(i, o, kernel_size=5, stride=2))
            else:
                decoder.append(nn.ConvTranspose2d(i, o, kernel_size=3, stride=1))
            decoder.append(nn.ReLU())
        for i, o in [(36, 24), (24, 3)]:
            decoder.append(nn.ConvTranspose2d(i, o, kernel_size=5, stride=2))
            decoder.append(nn.ReLU())
        self.decoder = nn.Sequential(*decoder)

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def run_sample(self, mu, logvar):
        std = torch.exp(logvar/2)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()
        return p, q, z

    def forward(self, x):
        x = self.encode(x)
        # return x
        mu, logvar = self.mu(x), self.var(x)
        p, q, z = self.run_sample(mu, logvar)
        return self.decode(z)
        # return self.decode(z), z, p, q
    
def loss_fn(y, x_hat, p, q):
    recon_loss = F.mse_loss(x_hat, y, reduction="mean")

    kl = torch.distributions.kl_divergence(q, p)
    kl = kl.mean()

    loss = kl * 0.4 + recon_loss
    return loss, kl, recon_loss

In [3]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = VAE(latent=25).to(device)
print(summary(model, (3, 66, 200)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 31, 98]           1,824
              ReLU-2           [-1, 24, 31, 98]               0
            Conv2d-3           [-1, 36, 14, 47]          21,636
              ReLU-4           [-1, 36, 14, 47]               0
            Conv2d-5            [-1, 48, 5, 22]          43,248
              ReLU-6            [-1, 48, 5, 22]               0
            Conv2d-7            [-1, 64, 3, 20]          27,712
              ReLU-8            [-1, 64, 3, 20]               0
            Conv2d-9            [-1, 64, 1, 18]          36,928
             ReLU-10            [-1, 64, 1, 18]               0
          Flatten-11                 [-1, 1152]               0
           Linear-12                 [-1, 1000]       1,153,000
             ReLU-13                 [-1, 1000]               0
           Linear-14                  [